In [ ]:
import os
import cv2
import numpy as np
from collections import defaultdict

import scikitplot
import seaborn as sns
from matplotlib import pyplot

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.layers import TimeDistributed, LSTM, Bidirectional
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
INPUT_PATH = "../input/ck48-5-emotions/CK+48/"
FER13_PATH = "../input/fer2013/dataset200224.xlsx"

for dir_ in os.listdir(INPUT_PATH):
    count = 0
    for f in os.listdir(INPUT_PATH + dir_ + "/"):
        count += 1
    print(f"{dir_} has {count} number of images")

In [ ]:
import pandas as pd

fer_data = pd.read_excel(FER13_PATH)
print(fer_data)
fer_data["emotion"].unique()

`sadness` and `fear` has very low number of images as compared to other classes

In [ ]:
FER13_EMOTIONS = ["anger",
                "disgust",
                "fear",
                "happy",
                "sadness",
                "surprise",
                "neutral"]
TOP_EMOTIONS = FER13_EMOTIONS

### Data Preprocessing

I first make the data compatible for neural nets

In [ ]:
def directimg2array(path):
    img = cv2.imread(path, 0)
    return img


In [ ]:
INPUT_PATH = "../input/ck48-5-emotions/CK+48/"

data = defaultdict(str)
for dir_ in os.listdir(INPUT_PATH):
    if dir_ in TOP_EMOTIONS:
        data[dir_] = []
        for f in os.listdir(INPUT_PATH + dir_ + "/"):
            data[dir_].append(f)
    else:
        data[dir_] = []

# data

In [ ]:
from keras.utils import to_categorical
def CRNO(df):
    df['pixels'] = df['pixels'].apply(lambda pixel_sequence: [int(pixel) for pixel in pixel_sequence.split()])
    data_X = np.array(df['pixels'].tolist(),dtype='float32').reshape(-1,48,48,1)/255.0
    data_Y = [int(emotion) for emotion in df["emotion"]]
    return data_X, data_Y

In [ ]:
print(fer_data["emotion"].value_counts())

In [ ]:
train_X,train_Y = CRNO(fer_data)

In [ ]:
for emotion in data:
    data[emotion] = [directimg2array(path = INPUT_PATH + emotion + "/"+v) for v in data[emotion]]

data["disgust"]=[]
data["neutral"]=[] 


In [ ]:
# reshape to merge with ck+
train_X = np.reshape(train_X,(len(train_X),48,48))

for index,fer in enumerate(train_X):
    data[FER13_EMOTIONS[train_Y[index]]]+=[fer]
print(data["happy"][0].shape)

In [ ]:
# min_samples = min(len(v) for v in data.values())

# for k, v in data.items():
#     if len(v) > min_samples:
#         data[k] = v[:min_samples]

for k, v in data.items():
    print(f"{k} has {len(v)} samples")
    print(v[0].shape)

In [ ]:
surprise = np.stack(data["surprise"], axis=0)
surprise = surprise.reshape(len(data["surprise"]),48,48,1)

happy = np.stack(data["happy"], axis=0)
happy = happy.reshape(len(data["happy"]),48,48,1)

anger = np.stack(data["anger"], axis=0)
anger = anger.reshape(len(data["anger"]),48,48,1)

sadness = np.stack(data["sadness"], axis=0)
sadness = sadness.reshape(len(data["sadness"]),48,48,1)

fear = np.stack(data["fear"], axis=0)
fear = fear.reshape(len(data["fear"]),48,48,1)

disgust = np.stack(data["disgust"], axis=0)
disgust = disgust.reshape(len(data["disgust"]),48,48,1)

neutral = np.stack(data["neutral"], axis=0)
neutral = neutral.reshape(len(data["neutral"]),48,48,1)

X = np.concatenate((anger, disgust, fear, happy,sadness, surprise, neutral))
y = np.concatenate((np.array([0]*len(data["anger"])), np.array([1]*len(data["disgust"])), np.array([2]*len(data["fear"])), np.array([3]*len(data["happy"])), np.array([4]*len(data["sadness"])),  np.array([5]*len(data["surprise"])),  np.array([6]*len(data["neutral"]))))
y = np_utils.to_categorical(y)

X.shape, y.shape

In [ ]:
print(happy.shape)

In [ ]:
# Create a list of integers from 0 to 6
indices = list(range(7))

# Map integers to emotions using list comprehension
label_encoder= [FER13_EMOTIONS[i] for i in indices]

label_encoder

In [ ]:
X_train, y_train, X_test, y_test = train_test_split(X, y, train_size=0.8, stratify=y, shuffle=True, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
import random
num = random.random()
np.random.seed(int(num*100))
anger_idx = np.random.choice(np.where(X_test[:, 0]==1)[0], size=1)
disgust_idx = np.random.choice(np.where(X_test[:, 1]==1)[0], size=1)
fear_idx = np.random.choice(np.where(X_test[:, 2]==1)[0], size=1)
happy_idx = np.random.choice(np.where(X_test[:, 3]==1)[0], size=1)
sad_idx = np.random.choice(np.where(X_test[:, 4]==1)[0], size=1)
surprise_idx = np.random.choice(np.where(X_test[:, 5]==1)[0], size=1)
neutral_idx = np.random.choice(np.where(X_test[:, 6]==1)[0], size=1)


fig = pyplot.figure(1, (6,13))
i = 0
for name, idx in zip(FER13_EMOTIONS, [anger_idx,disgust_idx, fear_idx, happy_idx, sad_idx, surprise_idx,neutral_idx]):
    i += 1
    ax = pyplot.subplot(5,3,i)
    sample_img = X_train[idx]
    sample_img = np.reshape(sample_img, (48,48))
    ax.imshow(sample_img, cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(name)

In [ ]:
# data normalization
X_train = X_train / 255.
X_test = X_test / 255.

In [ ]:
def build_dcnn(input_shape, show_arch=True):
    """
    This is a Deep Convolutional Neural Network (DCNN). For generalization purpose I used dropouts in regular intervals.
    I used `ELU` as the activation because it avoids dying relu problem but also performed well as compared to LeakyRelu
    atleast in this case. `he_normal` kernel initializer is used as it suits ELU. BatchNormalization is also used for better
    results.
    """
    net = Sequential(name='DCNN')

    net.add(
        Conv2D(
            filters=64,
            kernel_size=(3,3),
            input_shape=input_shape,
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_1'
        )
    )
    net.add(BatchNormalization(name='batchnorm_1'))
    net.add(
        Conv2D(
            filters=64,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_2'
        )
    )
    net.add(BatchNormalization(name='batchnorm_2'))
    
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_1'))
    net.add(Dropout(0.45, name='dropout_1'))

    net.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_3'
        )
    )
    net.add(BatchNormalization(name='batchnorm_3'))
    net.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_4'
        )
    )
    net.add(BatchNormalization(name='batchnorm_4'))
    
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_2'))
    net.add(Dropout(0.45, name='dropout_2'))

    net.add(
        Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_5'
        )
    )
    net.add(BatchNormalization(name='batchnorm_5'))
    net.add(
        Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_6'
        )
    )
    net.add(BatchNormalization(name='batchnorm_6'))
    
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_3'))
    net.add(Dropout(0.4, name='dropout_3'))

    net.add(
        Conv2D(
            filters=512,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_7'
        )
    )
    net.add(BatchNormalization(name='batchnorm_7'))
    net.add(
        Conv2D(
            filters=512,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_8'
        )
    )
    net.add(BatchNormalization(name='batchnorm_8'))
    
    net.add(Dropout(0.4, name='dropout_4'))
    
    net.add(GlobalMaxPool2D(name="globalmax2d"))
    
    if show_arch:
        net.summary()
    
    return net

In [ ]:
from tensorflow.keras.layers import Reshape, Lambda
import tensorflow.keras.backend as K

def memory_model(input_shape, num_class, show_arch=True):
    dcnn = build_dcnn(input_shape, show_arch=False)
    
    model = Sequential(name="convolutional_Bidrectional_LSTM")

    model.add(dcnn)
    model.add(Lambda(lambda x: K.reshape(x, (-1, 1, 512))))
    
    model.add(Bidirectional(LSTM(128, return_sequences=True, name="bidirect_lstm_1")))
    model.add(Dropout(.35, name="dropout_1"))
    model.add(Bidirectional(LSTM(64, return_sequences=False, name="bidirect_lstm_2")))
    model.add(Dropout(.45, name="dropout_2"))

    model.add(
        Dense(
            128,
            activation='elu',
            kernel_initializer='he_normal',
            name='dense_1'
        )
    )
    model.add(BatchNormalization(name='batchnorm_1'))
    model.add(Dropout(.7, name="dropout_3"))

    model.add(
        Dense(
            num_class,
            activation='softmax',
            name='out_layer'
        )
    )
    
    if show_arch:
        model.summary()
    
    return model

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.00005,
    patience=12,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.8,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
#     early_stopping,
    lr_scheduler,
]

batch_size = 128
epochs = 100

In [ ]:
INPUT_SHAPE = (48, 48, 1)
optim = optimizers.Nadam(0.001)

model = memory_model(INPUT_SHAPE, num_class=7)
model.compile(
        loss='categorical_crossentropy',
        optimizer=optim,
        metrics=['accuracy']
)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from keras.models import model_from_json

model.load_weights("/kaggle/input/model-68/model_ck_fer.h5")
history = model.fit(
    x=X_train,
    y=X_test,
    validation_data = (y_train,y_test),
    batch_size=batch_size,
    epochs=1000,
    callbacks=callbacks,
    use_multiprocessing=True
)

In [ ]:
sns.set()
fig = pyplot.figure(0, (12, 4))

ax = pyplot.subplot(1, 2, 1)
sns.lineplot(history.epoch, history.history['accuracy'], label='train')
sns.lineplot(history.epoch, history.history['val_accuracy'], label='valid')
pyplot.title('Accuracy')
pyplot.tight_layout()

ax = pyplot.subplot(1, 2, 2)
sns.lineplot(history.epoch, history.history['loss'], label='train')
sns.lineplot(history.epoch, history.history['val_loss'], label='valid')
pyplot.title('Loss')
pyplot.tight_layout()

pyplot.savefig('epoch_history.png')
pyplot.show()

`The fluctuations in the epoch metrics is due to the fact that we have very low data for such a complex task.`

In [ ]:
model.load_weights("/kaggle/input/model-68/model_ck_fer.h5")
yhat_valid = model.predict_classes(y_train)
scikitplot.metrics.plot_confusion_matrix(np.argmax(y_test, axis=1), yhat_valid, figsize=(7,7))
pyplot.savefig("confusion_matrix.png")

print(f'total wrong validation predictions: {np.sum(np.argmax(y_test, axis=1) != yhat_valid)}/{len(yhat_valid)}\n\n')
accuracy = ((np.sum(np.argmax(y_test, axis=1) != yhat_valid))/len(yhat_valid))*100
print(f'acurracy: {accuracy}\n\n')
print(classification_report(np.argmax(y_test, axis=1), yhat_valid))

`If we have more data to train then we will get better and more generalized model.`

In [ ]:
np.random.seed(0)
indices = np.random.choice(range(y_train.shape[0]), size=15, replace=False)

fig = pyplot.figure(1, (9,30))

i = 0
for idx in indices:
    true_emotion = FER13_EMOTIONS[np.argmax(y_test[idx])]
    pred_emotion = FER13_EMOTIONS[model.predict_classes(np.expand_dims(y_train[idx], axis=0))[0]]
    
    i += 1
    ax = pyplot.subplot(15,3,i)
    sample_img = y_train[idx,0,:,:,0]
    ax.imshow(sample_img, cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f"t:{true_emotion}, p:{pred_emotion}")

In [ ]:
import json
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
model.save_weights("model.h5")
print("saved model")